In [1]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import numpy as np
import os

from tqdm.notebook import tqdm

In [32]:
labels = pd.read_csv("./data/rsna-pneumonia-detection-challenge/train_labels.csv")
labels = labels.drop_duplicates("patientId")
labels

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30219,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,666.0,418.0,186.0,223.0,1
30221,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,609.0,464.0,240.0,284.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0


In [2]:
ROOT_PATH = Path("./data/rsna-pneumonia-detection-challenge/train")
TEST_PATH = Path("./data/rsna-pneumonia-detection-challenge/test")
SAVE_PATH = Path('./data/processed')

In [34]:
for c, patientid in enumerate(labels.patientId): 
    patient_id = labels.patientId.iloc[c]
    dcm_path = ROOT_PATH/patient_id
    dcm_path = dcm_path.with_suffix(".dcm")
    dcm = pydicom.read_file(dcm_path).pixel_array / 255
    
    dcm_array = cv2.resize(dcm, (224,224)).astype(np.float16)
    
    label = labels.Target.iloc[c]
    
    train_or_val = "train" if c < 24000 else "val"
    
    current_save_path = SAVE_PATH/train_or_val/str(label)
    current_save_path.mkdir(parents=True, exist_ok=True)
    np.save(current_save_path/patient_id, dcm_array)

In [7]:
for c, patientid in enumerate(os.listdir(TEST_PATH)): 
    dcm_path = TEST_PATH/patientid
    dcm_path = dcm_path.with_suffix(".dcm")
    dcm = pydicom.read_file(dcm_path).pixel_array / 255
    dcm_array = cv2.resize(dcm, (224,224)).astype(np.float16)
    
    current_save_path = SAVE_PATH/str("test")
    current_save_path.mkdir(parents=True, exist_ok=True)
    np.save(current_save_path/patientid[:-4], dcm_array)